In [1]:
from Maximus_optimized_non_dominated import Multi_Fair as maximus
from sklearn.model_selection import StratifiedShuffleSplit as ss
from DataPreprocessing.my_utils import get_score,get_fairness,vis

C:\Users\Arjun\Anaconda3\lib\site-packages\sklearn\utils\deprecation.py:143: FutureWarning: The sklearn.ensemble.forest module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.ensemble. Anything that cannot be imported from sklearn.ensemble is now part of the private API.
  warnings.warn(message, FutureWarning)
C:\Users\Arjun\Anaconda3\lib\site-packages\sklearn\utils\deprecation.py:143: FutureWarning: The sklearn.tree.tree module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.tree. Anything that cannot be imported from sklearn.tree is now part of the private API.
  warnings.warn(message, FutureWarning)
C:\Users\Arjun\Anaconda3\lib\site-packages\sklearn\utils\deprecation.py:143: FutureWarning: The sklearn.neighbors.base module is  deprecated in version 0.22 and will be removed in version 0.24. 

In [2]:
 def train_classifier1(X_train, X_test, y_train, y_test, sa_index, p_Group, base_learners,preference):
    
    classifier = maximus(n_estimators=base_learners, saIndex=sa_index, saValue=p_Group,preference=preference)
    
    classifier.fit(X_train, y_train)
    
    y_pred_probs = classifier.predict_proba(X_test)[:, 1]
    y_pred_labels = classifier.predict(X_test)
    f=classifier.feature_importances_
    #return classifier.conf_scores, classifier.get_weights_over_iterations(), classifier.get_initial_weights()
    return y_pred_probs, y_pred_labels,classifier


In [12]:
def train(X,y):
    in_ts,pred1,fx=[],[],[]
    sss = ss(n_splits=5,test_size=0.4)    
    for train_index, test_index in sss.split(X, y):
        X_train, X_test = X[train_index], X[test_index]
        y_train, y_test = y[train_index], y[test_index]
        in_ts.append(test_index)    
        pb1,pd1,f1=train_classifier1(X_train, X_test, y_train, y_test, sa_index,     p_Group, 499,[0.33,0.34,0.33])
        pred1.append(pd1)
        fx.append(f1)
        print(f1.theta-1," : ",f1.ob[f1.theta-1])
    return in_ts,pred1,fx

In [13]:
results,performance,Hx=[],[],[]
for dt in ['Compas']:
    if dt=='Adult':
        from DataPreprocessing.load_adult import load_adult
        X, y, sa_index, p_Group, x_control,F = load_adult()
        #v='Adult_2_sensi_Mari_Sex'
        saf=sa_index[1]
    elif dt=='Bank':
        from DataPreprocessing.load_bank import load_bank
        X, y, sa_index, p_Group, x_control,F = load_bank()
        saf=sa_index[0]
        print(saf)
    elif dt=='Credit':
        from DataPreprocessing.load_credit import load_credit
        X, y, sa_index, p_Group, x_control,F = load_credit()
        saf=sa_index[0]
    elif dt=='Compas':
        from DataPreprocessing.load_compas_data import load_compas
        X, y, sa, p_G, x_control,F = load_compas()
        sa_index=[sa[-1],sa[0]]
        p_Group=[p_G[-1],p_G[0]]
    sensitives=[F[v] for v in sa_index]
    in_ts,pred1,f1=train(X,y)
    results.append(list(get_fairness(sa_index,p_Group,in_ts,pred1,X,y).values()))
    performance.append(get_score(pred1,in_ts,X,y))
    Hx.append(f1)

Features we will be using for classification are: ['age_cat_25 - 45', 'age_cat_Greater than 45', 'age_cat_Less than 25', 'race', 'sex', 'priors_count', 'c_charge_degree', 'target'] 

going
105  :  [0.34491472 0.00834023 0.04984424]
going
105  :  [0.33891346 0.01602889 0.08934119]
going
117  :  [0.3357549  0.06585323 0.09609883]
going
139  :  [0.34080859 0.02974987 0.06660936]
going
77  :  [0.33986102 0.09135569 0.04850976]


For Sensitive attribute index  4
avg_TPR_unprot: 0.6751112767806579 avg_TPR_prot: 0.68829321840291
avg_TNR_unprot: 0.6383183359615673 avg_TNR_prot: 0.622128846141885

-------------------------------------------



For Sensitive attribute index  3
avg_TPR_unprot: 0.6428515011820437 avg_TPR_prot: 0.6947004480755558
avg_TNR_unprot: 0.6287279783443929 avg_TNR_prot: 0.6394380538245288

-------------------------------------------

avg_TPR: 0.6770623742454729 avg_TNR: 0.634525939177102
avg_acc: 0.6545454545454545 avg_Bacc: 0.6557941567112874
avg_auc: 0.6557941567112875 av

In [15]:
Pareto_Fronts=[Hx[i].PF.values() for i in range(len(Hx))]

In [16]:
ALL_solutions=[Hx[i].ob for i in range(len(Hx))]

In [ ]:
pseudo_weights=[Hx[i].pseudo_weights for i in range(len(Hx))]